# Adam Thomson - PHY573 - Week 3

## Deutsch-Jozsa Algorithm

The goal of the Deutsch-Jozsa Algorithm is to answer the question of whether a given oracle function is "constant" or "balanced". In other words, the algorithm will take an input function that always returns either the same single-bit answer or has a 50/50 chance of returning either a 0 or 1, and will determine which category the function is in. Classically, the best case scenario is querying the function twice and receiving different values to identify a balanced function; in the worst case it must query the function for more than half of all possible inputs to ensure it is constant. However, if the input function is implemented on a quantum computer, then the answer can be determined in a single query every time! Let's take a closer look at how this is possible by following the qiskit notebook https://github.com/Qiskit/textbook/blob/main/notebooks/ch-algorithms/deutsch-jozsa.ipynb

In [4]:
# Import libraies
from IPython.display import Math
from qiskit import \
    transpile, \
    QuantumCircuit as QCir, \
    QuantumRegister as QReg, \
    ClassicalRegister as CReg
from qiskit.circuit.library import Initialize
from qiskit.result import marginal_counts
from qiskit.quantum_info import random_statevector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import \
    plot_histogram, \
    plot_bloch_multivector, \
    array_to_latex
from qiskit_ibm_runtime import \
    QiskitRuntimeService, \
    SamplerV2

from qiskit_aer import AerSimulator

### Inspect the Quantum Solution

In [5]:
# We start with an initial state of a quantum register with as many qubits as the oracle function
#  takes as input, n, in |0> and an additional qubit initialized to |1>

display(Math(r'\ket{\psi_0} = \ket0 ^{\otimes n} \ket1'))

<IPython.core.display.Math object>

In [10]:
# We then apply a Haddamard gate to each qubit and simplify the state expression

display(Math(r'\ket{\psi_1} = H^{\otimes n + 1}\ket{\psi_0}'))
display(Math(r'\ket{\psi_1} = H^{\otimes n} \ket0^{\otimes n} \otimes H\ket1'))

display(Math(r'''
    H^{\otimes n} \ket0 = H\ket0 \otimes H\ket0 ... \otimes H\ket0
\\\\\qquad\qquad   = \frac1{\sqrt2}(\ket0 + \ket1) \otimes ... \otimes \frac1{\sqrt2}(\ket0 + \ket1)
\\\\\qquad\qquad   = \frac1{\sqrt{2^n}} \Big( \ket0 + \ket1 + \ket2 + ... + \ket{2^n - 1}  \Big)
\\\\\qquad\qquad   = \frac1{\sqrt{2^n}} \sum_{x=0}^{2^n -1} \ket x
'''))

display(Math(r'''
    \ket{\psi_1} = H^{\otimes n} \ket0 \otimes H\ket1
\\\\\qquad = \frac1{\sqrt{2^n}} \sum_{x=0}^{2^n -1} \ket x \otimes (\frac1{\sqrt2})(\ket0 - \ket1)
'''))

display(Math(r'''
    \ket{\psi_1} = \frac1{\sqrt{2^{n + 1}}} \sum_{x=0}^{2^n -1} \ket x(\ket0 - \ket1)
'''))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
# Apply the oracle function
display(Math(r'''
\textnormal{Next, apply the oracle function on } \ket{\psi_1} \textnormal{ such that } f(\ket x \ket y) = \ket x \ket{y \oplus f(x)}
'''))

display(Math(r'''
\ket{\psi_2} = f \big( \frac1{\sqrt{2^{n + 1}}} \sum_{x=0}^{2^n -1} \ket x(\ket0 - \ket1) \big)
\\\\ \qquad = \frac1{\sqrt{2^{n + 1}}}\sum_{x=0}^{2^n -1} (f\ket x (\ket0 - \ket1))
\\\\ \qquad = \frac1{\sqrt{2^{n + 1}}}\sum_{x=0}^{2^n -1} \ket x (\ket{0 \oplus f(x)} - \ket{1 \oplus f(x)})
'''))

display(Math(r'''
\forall x: f(x) = \{0,1\}
'''))

display(Math(r'''
\textnormal{When }f(x)=0:
\\\\ \big(\ket{0 \oplus f(x)} - \ket{1 \oplus f(x)}\big) = (\ket0 - \ket1) = (-1)^{f(x)}(\ket0 - \ket1)
'''))

display(Math(r'''
\textnormal{When }f(x)=1: 
\\\\ \big(\ket{0 \oplus f(x)} - \ket{1 \oplus f(x)}\big) = (\ket1 - \ket0) = (-1)^{f(x)}(\ket0 - \ket1)
'''))

display(Math(r'''
\therefore
\\\\ \ket{\psi_2} = \frac1{\sqrt{2^{n+1}}} \sum_{x=0}^{2^n -1} (-1)^{f(x)} \ket x(\ket0 - \ket1)
'''))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [8]:
# Now, we ignore the second register and apply a Hadamard gate on all qubits in the first register again

display(Math(r"\ket{\psi_{2'}} = \frac1{\sqrt{2^n}} \sum_{x=0}^{2^n -1} (-1)^{f(x)} \ket x"))
display(Math(r'''
\ket{\psi_3} = H^{\otimes n}\ket{\psi_{2'}}
\\\\\qquad = H^{\otimes n}(\frac1{\sqrt{2^n}} \sum_{x=0}^{2^n -1} (-1)^{f(x)} \ket x)
\\\\\qquad = \frac1{\sqrt{2^n}} \sum_{x=0}^{2^n -1} (-1)^{f(x)} H^{\otimes n} \ket x
'''))

display(Math(r'''
\textnormal{For each individual qubit,} \ket{x_i}
\\\\ H\ket{x_i} = \frac1{\sqrt2}(\ket0 + (-1)^{x_i} \ket 1)
'''))

display(Math(r'''
\textnormal{To help simplify, define the bitwise product operation}
\\\\
x \cdot y = x_0y_0 \oplus x_1y_1 \oplus ... \oplus x_{n-1}y_{n-1}
'''))

display(Math(r'''\textnormal{So when summed over all qubits, we have}
\\\\ H^{\otimes n}\ket x = \frac{(-1)^{x\cdot0}}{\sqrt{2^n}} \ket0 + \frac{(-1)^{x\cdot1}}{\sqrt{2^n}} \ket1 + ... \frac{(-1)^{x\cdot{2^n-1}}}{\sqrt{2^n}} \ket{2^n-1}
\\\\\qquad\qquad = \frac1{\sqrt{2^n}} \sum_{y=0}^{2^n-1} (-1)^{x \cdot y} \ket y
'''))

display(Math(r'''
\textnormal{Going back to the original operation, we get equations matching the book}
\\\\ \ket{\psi_3} = H^{\otimes n}\ket{\psi_{2'}}
\\\\\qquad = \frac1{\sqrt{2^n}} \sum_{x=0}^{2^n-1}(-1)^{f(x)} H^{\otimes n} \ket x
\\\\\qquad = \frac1{\sqrt{2^n}} \sum_{x=0}^{2^n-1}(-1)^{f(x)} \big[ \frac1{\sqrt{2^n}} \sum_{y=0}^{2^n-1} (-1)^{x \cdot y} \ket y \big]
\\\\\qquad = \frac1{2^n} \sum_{x=0}^{2^n-1}(-1)^{f(x)} \big[ \sum_{y=0}^{2^n-1} (-1)^{x \cdot y} \ket y \big]
\\\\\qquad = \frac1{2^n} \sum_{y=0}^{2^n-1} \Big [ \sum_{x=0}^{2^n-1} (-1)^{f(x)} (-1)^{x \cdot y} \Big] \ket y
'''))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [9]:
# Discuss conclusions
display(Math(r'''
\textnormal{By measuring the state now, the oracle function can be determined to be constant or balanced.}
'''))
display(Math(r'''
\textnormal{Consider the case when the oracle function is constant, or f(x) is always one of 0 or 1.}
\\\\ \textnormal{ - If f(x) = 0, then the term } (-1)^{f(x)} \textnormal{ had no effect on the transformation and } H(H\ket0) = \ket0
\\\\ \textnormal{ - If f(x) = 1, then the term } (-1)^{f(x)} \textnormal{ applied a phase flip to all qubits but did not effect measurement probabilities}
\\\\ \textnormal{Both of these scenarios always lead to a measurement of } \ket0^{\otimes n}
'''))
display(Math(r'''
\\\\ \textnormal{Otherwise, the oracle function is balanced and will produce a 0 or 1 with equal probability across all inputs.}
\\\\ \textnormal{ - In this case, half of the qubits in the register have their phase flipped. This creates a state orthogonal to } \ket{00..0}
\\\\ \textnormal{Meaning a balanced oracle will never result in a measurement of } \ket0^{\otimes n}
'''))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Example Circuit